# Wide and Deep Regressor

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# download dataset
housing = fetch_california_housing()

In [4]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [5]:
# scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [6]:
# build model
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)  # connects the input with the first hidden layer
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)  # connects both hidden layers
concat = keras.layers.Concatenate()([input_, hidden2])  # connects the input with the second hidden layer into a new concatenated layer
output = keras.layers.Dense(1)(concat)  # connects the concatenated layer to the output
model = keras.Model(inputs=[input_], outputs=[output])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-20 16:30:48.530283: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-20 16:30:48.530405: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# compile
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(lr=0.001))

/Users/sebastian/Documents/GitHub/study/study_env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [8]:
# fit
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


2023-02-20 16:30:51.378313: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-20 16:30:51.511314: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.7519

2023-02-20 16:30:53.396777: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 2s 6ms/step - loss: 1.7519 - val_loss: 0.7824
Epoch 2/20
363/363 [==============================] - 2s 6ms/step - loss: 0.7388 - val_loss: 0.6724
Epoch 3/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6809 - val_loss: 0.6314
Epoch 4/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6187 - val_loss: 0.5873
Epoch 5/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5821 - val_loss: 0.5583
Epoch 6/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5525 - val_loss: 0.5312
Epoch 7/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5345 - val_loss: 0.5162
Epoch 8/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5110 - val_loss: 0.4952
Epoch 9/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5050 - val_loss: 0.4871
Epoch 10/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4847 - val_loss: 0.4725
Epoch 11/2

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 38)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

In [10]:
# evaluate
mse_test = model.evaluate(X_test, y_test)
mse_test

162/162 [==============================] - 1s 3ms/step - loss: 0.4191


0.4190855026245117

In [11]:
# predictions on "new" instances
X_new = X_test[:3]
y_pred = model.predict(X_new)
y_pred

1/1 [==============================] - 0s 183ms/step


2023-02-20 16:31:57.838327: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[2.068304  ],
       [3.0843227 ],
       [0.80690134]], dtype=float32)

# Wide and Deep with Two Input Layers

In [13]:
# build a model using the functional API
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, input_B])  # another approach to make a concatenate layer
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [19]:
# compile
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(learning_rate=0.001))

In [6]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [21]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
 29/363 [=>............................] - ETA: 1s - loss: 7.2971

2023-02-21 09:16:08.172229: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 4.4315

2023-02-21 09:16:09.715381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 2s 5ms/step - loss: 4.4315 - val_loss: 2.0352
Epoch 2/20
363/363 [==============================] - 2s 5ms/step - loss: 1.4141 - val_loss: 1.0222
Epoch 3/20
363/363 [==============================] - 2s 4ms/step - loss: 0.8659 - val_loss: 0.7795
Epoch 4/20
363/363 [==============================] - 2s 4ms/step - loss: 0.7238 - val_loss: 0.6929
Epoch 5/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6702 - val_loss: 0.6522
Epoch 6/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6396 - val_loss: 0.6263
Epoch 7/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6184 - val_loss: 0.6060
Epoch 8/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6018 - val_loss: 0.5906
Epoch 9/20
363/363 [==============================] - 2s 4ms/step - loss: 0.5887 - val_loss: 0.5781
Epoch 10/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5781 - val_loss: 0.5686
Epoch 11/2

In [22]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 wide_input (InputLayer)        [(None, 5)]          0           []                               
                                                                                                  
 deep_input (InputLayer)        [(None, 6)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 11)           0           ['wide_input[0][0]',             
                                                                  'deep_input[0][0]']             
                                                                                                  
 output (Dense)                 (None, 1)            12          ['concatenate_1[0][0]']    

In [30]:
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
mse_test

162/162 [==============================] - 1s 3ms/step - loss: 0.5162


0.5161593556404114

In [31]:
y_pred = model.predict((X_new_A, X_new_B))
y_pred

1/1 [==============================] - 0s 28ms/step


2023-02-21 09:20:10.115062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[1.9343538],
       [2.912988 ],
       [1.0713353]], dtype=float32)

# Wide and Deep with Auxiliary Output

In [7]:
# build a model using the functional API
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, input_B])  # another approach to make a concatenate layer
output = keras.layers.Dense(1, name="main_output")(concat)  # main output
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)  # auxiliary output
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-21 09:46:55.301708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 09:46:55.301866: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
# use MSE as loss function for both outputs
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

In [9]:
# both the main and auxiliary output will try to predict the same data
history = model.fit([X_train_A, X_train_B], [y_train, y_train],
                    epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))


Epoch 1/20


2023-02-21 09:46:58.717118: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/363 [..............................] - ETA: 2:16 - loss: 8.1047 - main_output_loss: 8.4504 - aux_output_loss: 4.9933

2023-02-21 09:46:58.934069: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.3016 - main_output_loss: 1.2021 - aux_output_loss: 2.1968

2023-02-21 09:47:01.737736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 3s 9ms/step - loss: 1.3016 - main_output_loss: 1.2021 - aux_output_loss: 2.1968 - val_loss: 0.6376 - val_main_output_loss: 0.5529 - val_aux_output_loss: 1.3999
Epoch 2/20
363/363 [==============================] - 3s 8ms/step - loss: 0.6504 - main_output_loss: 0.5755 - aux_output_loss: 1.3248 - val_loss: 0.5797 - val_main_output_loss: 0.5074 - val_aux_output_loss: 1.2298
Epoch 3/20
363/363 [==============================] - 3s 8ms/step - loss: 0.6196 - main_output_loss: 0.5574 - aux_output_loss: 1.1793 - val_loss: 0.5531 - val_main_output_loss: 0.4918 - val_aux_output_loss: 1.1049
Epoch 4/20
363/363 [==============================] - 3s 8ms/step - loss: 0.6289 - main_output_loss: 0.5814 - aux_output_loss: 1.0564 - val_loss: 0.5425 - val_main_output_loss: 0.4925 - val_aux_output_loss: 0.9922
Epoch 5/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5880 - main_output_loss: 0.5474 - aux_output_loss: 0.9533 - val_loss: 0.5350 - 

In [10]:
# get total and individual loss of the output layers
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])
print(total_loss, main_loss, aux_loss)


162/162 [==============================] - 1s 5ms/step - loss: 0.5352 - main_output_loss: 0.5245 - aux_output_loss: 0.6317
0.5352488160133362 0.5245314240455627 0.631702721118927


In [11]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])
print(y_pred_main, y_pred_aux)

1/1 [==============================] - 0s 59ms/step
[[2.2234037]
 [1.4197128]
 [1.3990589]] [[2.4236834]
 [1.3715186]
 [1.0291361]]


2023-02-21 09:47:58.231225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [13]:
# save and load the model
model.save("my_model.h5")
model = keras.models.load_model("my_model.h5")

# Callbacks

In [14]:
# build and compile a model
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)  # connects the input with the first hidden layer
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)  # connects both hidden layers
# connects the input with the second hidden layer into a new concatenated layer
concat = keras.layers.Concatenate()([input_, hidden2])
# connects the concatenated layer to the output
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])
# compile
model.compile(loss='mean_squared_error', optimizer="sgd")


In [ ]:
# save the partially trained model everytime an epoch has passed
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model.h5")
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_cb])

In [ ]:
# save the partially trained model everytime an epoch has passed only if the validation test is performing better than before
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model.h5", save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_cb])
model = keras.models.load_model("my_model.h5")  # roll back to the best model

In [ ]:
# stop the fitting if the validation test performance hasnt improved in 10 epochs
# also, save the model after each epoch
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, 
                                                  restore_best_weights=True)  # after the fitting is early stopped, revert the weights to the best ones
history = model.fit(X_train, y_train, 
                    epochs=10,  # now this can be very big because the callback will stop the training anyway
                    callbacks=[checkpoint_cb, early_stopping_cb])
